Preprocessing
-> open this notebook in google colab

In [1]:
#connect to gdrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!ls "/content/drive/My Drive/Colab Notebooks"

 data   data_preparer.py  'H&M.ipynb'   __pycache__   train.ipynb


In [3]:
#imports
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm
import sys
sys.path.insert(0,"/content/drive/My Drive/Colab Notebooks/")
import data_preparer
from pathlib import Path
from keras.utils import to_categorical

In [4]:
#prepare mask for vectors
rel = data_preparer.get_vectors_relative_position(7, 180, 20)
abss = data_preparer.get_vectors_absolute_position(rel)
mask = data_preparer.get_mask(abss)

Spacing: 30.0 degrees


In [5]:
#read recon file
file = "sub001_2drt_07_grandfather1_r1_recon.h5"
filepath = f"/content/drive/My Drive/Colab Notebooks/data/sub001/2drt/recon/{file}"
recon = data_preparer.get_recon(filepath)

In [6]:
#read timestamps file
filepath = "/content/drive/My Drive/Colab Notebooks/data/sub001/timestamps/sub001_2drt_01_vcv1_r1_recon.csv"
timestamps = data_preparer.read_timestamps(filepath)

In [7]:
#get vectors of the first row of timestamps
vectors = data_preparer.get_pixel_data(recon, abss, timestamps.iloc[0])

In [16]:
all_vectors = np.empty(shape=(7,20,52))
all_vectors.shape

(7, 20, 52)

In [22]:
#TO DO
#now we need to get all vectors for every row of timestamps - but pixel data is different size...
all_vectors = np.zeros(shape=(36,7,20,52)) # we need 36 arrays with 7 vectors of pixel data
for i in range(len(timestamps)):
    vectors = data_preparer.get_pixel_data(recon, abss, timestamps.iloc[i])
    print(i, vectors.shape)
    all_vectors[i] = vectors


0 (7, 20, 52)
1 (7, 20, 40)


ValueError: ignored

In [22]:
#and the label/letter of the vectors
timestamps['Buchstabe'] = timestamps['Buchstabe'].astype('category') #convert to catergory type
timestamps['letter_cat'] = timestamps["Buchstabe"].cat.codes # get new col with numeric category

In [ ]:
X = all_vectors#vectors
y = timestamps['letter_cat']#letter

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)

Start the training...

Used: https://medium.com/@dclengacher/keras-lstm-recurrent-neural-networks-c1f5febde03d & https://shap.readthedocs.io/en/latest/example_notebooks/text_examples/sentiment_analysis/Keras%20LSTM%20for%20IMDB%20Sentiment%20Classification.html

In [25]:
from keras.models import Model
from keras.layers import Input, Dense, TimeDistributed
from keras.layers import LSTM, Bidirectional, Conv1D, concatenate, Permute, Dropout

In [ ]:
#set up the model
model = Sequential()
model.add(LSTM(8,input_shape=(5,1),return_sequences=False))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=15,
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)